# 強化学習入門 

- **[1.1 強化学習とは](#1.1-強化学習とは)**
    - **[1.1.1 機械学習の分類](#1.1.1-機械学習の分類)**
    - **[1.1.2 強化学習というタスク](#1.1.2-強化学習というタスク)**
    - **[1.1.3 N腕バンディット問題](#1.1.3-N腕バンディット問題)**
    - **[1.1.4 エージェントの作成](#1.1.4-エージェントの作成)**
    - **[1.1.5 環境の作成](#1.1.5-環境の作成)**
    - **[1.1.6 報酬の定義](#1.1.6-報酬の定義)**
    - **[1.1.7 まとめ](#1.1.7-まとめ)**
<br><br>
- **[1.2 N腕バンディッド問題における方策](#1.2-N腕バンディッド問題における方策)**
    - **[1.2.1 greedy手法](#1.2.1-greedy手法)**
    - **[1.2.2 greedy手法](#1.2.2-greedy手法)**
    - **[1.2.3 楽観的初期値法](#1.2.3-楽観的初期値法)**
    - **[1.2.4 soft-max法](#1.2.4-soft-max法)**
    - **[1.2.5 UCB1アルゴリズム](#1.2.5-UCB1アルゴリズム)**
    - **[1.2.6 まとめ](#1.2.6-まとめ)**
    - **[1.2.7 探索と利用のトレードオフ](#1.2.7-探索と利用のトレードオフ)**
<br><br>
- **[1.3 添削問題](#1.3-添削問題)**


***

## 強化学習とは

###  1.1.1機械学習の分類

機械学習は主に3つの分野に分かれます。

- **<font color=#AA0000>教師あり学習</font>**<br>
**正解ラベル付きのトレーニングデータからモデルを学習し、未知のデータに対して予想することを目的とします。** 教師あり学習は以下の２つに分類されます。

    - 分類問題(コンテンツ:<a href="https://aidemy.net/courses/5010" target="_blank">[教師あり学習(分類)]</a>)<br>
     カテゴリ別に分けてあるデータを学習し、未知のデータのカテゴリ(離散値)を予測します。
     古典的な学習方法しては、SVMや決定木、ロジスティック回帰などがあり、deepな手法としては、VGG16などをファインチューニングした画像識別モデルなどがあります。
       
    - 回帰問題(コンテンツ:<a href="https://aidemy.net/courses/5010" target="_blank">[教師あり学習(回帰)]</a>)<br>
      分類問題と違って、こちらは連続値を予測します。株価の予測などはこちらに分類されます。
      

- **<font color=#AA0000>教師なし学習</font>**(コンテンツ:<a href="https://aidemy.net/courses/5030" target="_blank">[教師なし学習]</a>)<br>
**正解ラベルのないデータや構造が不明なデータに対し、データの構造や関係性を見出すことを目的とします。**  古典的なアルゴリズムとしては、主成分分析やk-means法などがあげられます。deepな手法としては、オートエンコーダや生成モデル、制約的ボルツマンマシンなどがあげられます。

- **<font color=#AA0000>強化学習</font>**<br>
**エージェントと環境が相互作用するような状況下で、もっとも最適な行動を発見することを目的とします。** 強化学習は、前提条件からゲームと相性が良いです。ゲームにおいていえば強化学習によって「勝ち方を発見すること」ができます。このコンテンツでは、この強化学習の基礎的な手法をコードを書きながら学びます。強化学習の理論部分だけを理解したいという場合は、実装クイズを飛ばして進めていくと良いでしょう。


#### 問題

次のうち、正しく述べている文章を選択して下さい。


- 教師あり学習は、学習データに対して予測を行う。
- 教師なし学習は、正解ラベル付きのデータを元に学習を行う。
- 強化学習は、エージェントと環境が相互作用するような状況下で、エージェントのもっとも最適な行動を発見することである。
- 上記の全て

#### ヒント

文書を読んで、機械学習の３つの分野の意味を理解しましょう。

#### 解答

- 強化学習は、エージェントと環境が相互作用するような状況下で、エージェントのもっとも最適な行動を発見することである。

*****

### 1.1.2強化学習というタスク

強化学習では、下の図のような**エージェント**と**環境**が相互作用する状況を前提に考えます。

<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/1535038969792547.png">
<center>図1.1.2 強化学習が前提としている状況</center>

ある**エージェント**が、**状態**$s$にあるとして、環境に対して**行動**$a$を取ります。そうすると、**環境**はその行動の評価として**報酬**$R$を返し、エージェントは次の状態$s'$へと移ります。これを繰り返し**行動を強化しながら**タスクを進めます。

強化学習において、上図では報酬(**即時報酬**)のみ最大化しようとしていますが、本来は、即時報酬だけでなく、そのあとに得られる報酬(**遅延報酬**)も含めた「**収益**」を最大化することが必要になります。<br>
例えば、株の売買により利益(または損失)を得る問題は強化学習にあたります。ここでの報酬は、売買する度の利益をさします。持っている株をすべて売り出せば確かにその時点では最もキャッシュを得ることができますが、より長期的な意味での価値を最大化するには、株をもう少し手元に置いておいたほうが良いかもしれません。このように、「未来の報酬の総和を最大化するための行動を選択する」ことを目的とします。

強化学習の例:<br>
・ブロック崩しでスコアをあげる<br>
・ビットコインを取引して増やす<br>
・安全に駐車をする<br>
・ロボットを歩かせる<br>
など<br>

つまり、**本質的な将来の価値を最大化することを目的とした問題は「強化学習に適したタスク」** と言えます。<br>

強化学習が今注目を浴びている理由は、それぞれの行動に対して評価することが難しい場合(不確実性のある環境)において真価を発揮するためであり、ゲームやロボットの自動制御、ファイナンス等で利用されています。

#### 問題

次の選択肢から強化学習を用いるべき事例を選んでください。

- 手書きの数字と実際の数字が結びつけられたデータで学習し、手書きの数字を見せられた時にそれがどの数字を表しているのかを出力する。
- 複数のマイクで取得された音声データで学習し、音の発信源ごとにクラスタリングを行う。
- ロボットアームで物を掴むという環境下で試行錯誤したデータから、最適なアームの制御法を獲得する。
- データから特徴を類推する。

#### ヒント

- 強化学習は行動に対して評価をすることが難しいものに適しています。

#### 解答

- ロボットアームで物を掴むという環境下で試行錯誤したデータから、最適なアームの制御法を獲得する。

***

### 1.1.3 N腕バンディット問題

N腕バンディット問題(多腕バンディット問題)とは、強化学習のもっとも簡単な理論を理解するのに適した環境と、
ここからは例題として **<font color=#AA0000>N腕バンディッド問題</font>** という問題を扱いながら強化学習についての理解を深めていきましょう。

**N腕バンディッド問題** とは、  

* 事前に確率の定義されたスロットマシーンがN台ならんでいる。(ユーザーはどのスロットマシーンがどのくらい当たるかは事前に知らされていない)  

* 一度スロットマシーンを引くと、それぞれ事前に設定されていた確率に基づき、当たりならば1、外れならば0  
という形で報酬が支払われる。(簡単化のため、今回はスロットの当たりの確率は変わらない)  

* ユーザーは1回の試行につきどれか1つのスロットマシーンを引くことができる。  


<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/1537893450430895.png">
<center>図1.1.3 N腕バンディット問題</center>

このような環境下で **試行回数あたりの平均報酬量を最大化する** ためにはどのようにするべきかを考える問題の事です。当然、一番確率の大きいスロットマシーンを引き続けるのが理にかなっていますが、エージェント自身はそれぞれの内部の確率を知らないために、実際に引くことによって得た報酬量からそれぞれの確率を推測しなければならないことが重要な点となっています。


ここから、この**N腕バンディッド問題**を題材に実装していきながら、強化学習について勉強していきます。

#### 問題

- 強化学習において、次の選択肢から正しいものを選んでください。

- 報酬を最大にするために、あたりの出る確率の高いと思われるスロットのみを引いたほうがいい。
- 確率がわからないのですべてのマシンを均等に引いた。
- 確率が明らかに低そうなマシンはあまり引くべきではない。
- すべて誤りである。

#### ヒント

- 強化学習では利益を出すためにいい確率のものを多くひくことはもちろん大切ですが、確率のいいものを探すために試行することも大切であり、極端に片方に偏るモデルはいいモデルとは言えません。

#### 解答

- 確率が明らかに低そうなマシンはあまり引くべきではない。

***

### 1.1.4 エージェントの作成

 **<font color=#AA0000>エージェント</font>** とは、**環境の中で行動を決定し、環境に対して影響を与えるもの**を指しています。N腕バンディット問題では、どのスロットマシーンを使用するかを**判断**し**報酬を受け取り**、**次の判断をする**ユーザーが **<font color=#AA0000>エージェント</font>** に該当します。<br>
 
 取得した報酬から、どのようなアルゴリズムに基づいて次の腕を決めるかという指標のことを **<font color=#AA0000>方策</font>** と言います。例えばN腕バンディッド問題において方策が「常に0」なのであれば、エージェントは常に0の腕を引き続けることになります。
 
この **エージェント** の最適な **<font color=#AA0000>方策</font>** を決定するのかが、この章の目標です。

#### 問題

- 5個のスロットマシーンの中から次に引くマシーンを**ランダムに**選択するエージェントを実装して下さい。
- 0〜4の数字をランダムに返す関数を実装しましょう。

In [ ]:
import random
import numpy as np

# 再現性を出すためにseedを設定します
random.seed(0)

# 関数random_selectを作ってください
def random_select():
    slot_num = 
    return slot_num


# 関数の呼び出し
slot_num = random_select()
print(slot_num)


#### ヒント

- `random.randint()`関数などを使用しましょう。

#### 解答例

In [ ]:
import random
import numpy as np

# 再現性を出すためにseedを設定します
random.seed(0)

# 関数random_selectを作ってください
def random_select():
    slot_num = np.random.randint(0, 5)
    return slot_num


# 関数の呼び出し
slot_num = random_select()
print(slot_num)


***

### 1.1.5 環境の作成

 **<font color=#AA0000>環境</font>** とは、エージェントが行動をおこす対象です。エージェントの行動を受け、状況を観測し、報酬をエージェントに送信し、時間を1つ進めるという役割があります。
 
N腕バンディッド問題においては、エージェントがあるスロットマシーンを引いた時、そのスロットの確率によって当たりか外れかを出すプロセスに該当します。

NumPyの`numpy.random.binomial(n, p)`で、試行回数n、確率pの二項分布を求めることが出来ます。
```python
# 二項分布の計算（試行回数：1000, 確率：0.5, サンプル数：5）
x = np.random.binomial(1000, 0.5, 5)
print(x)
出力:
[489 514 497 499 519]
```

#### 問題

N腕バンディッド問題の**環境**を関数で実装して下さい。   
スロットマシーンは全部で5台あり、それぞれ当たりが出る確率は0.3, 0.4, 0.5, 0.6, 0.7とします。  
引数は、引くスロットマシーンの番号(0,1,2,3,4)とします。    

In [ ]:
import random
import numpy as np

random.seed(0)
np.random.seed(0)

# 関数environmentsを作ってください
def environments(band_number):
    # 各マシンの確率をNumPy配列にしてください
    coins_p = 
    
    # それぞれの確率で結果(0か1)を求めてください
    results = 
    
    #腕の番号がband_numberの結果を出力してください
    result = 
    
    return result

# 関数の呼び出し
band_number = 1
result = environments(band_number)

print(result)


#### ヒント

- Numpy配列にするには、`np.array()`を使います。
- `result` には指定したスロットが当たりか外れか(1 か 0)を返してください。

#### 解答例

In [ ]:
import random
import numpy as np

random.seed(0)
np.random.seed(0)

# 関数environmentsを作ってください
def environments(band_number):
    # 各マシンの確率をNumPy配列にしてください
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    
    # それぞれの確率で結果(0か1)を求めてください
    results = np.random.binomial(1, coins_p)
    
    #腕の番号がband_numberの結果を出力してください
    result = results[band_number]
    
    return result

# 関数の呼び出し
band_number = 1
result = environments(band_number)

print(result)


***

### 1.1.6 報酬の定義

 **<font color=#AA0000>報酬(reward)</font>** とは、環境からエージェントに与えられる信号のことで、**エージェントの一連の行動の望ましさを評価する指標**の事です。 N腕バンディッド問題において言えば**スロットマシーンから得られた返り値**がそのまま該当します。また、引いた直後(行動の直後)に得られた報酬を**即時報酬**と言います。

#### 問題

- 先ほどの環境関数を使用し、time番目を実行したN腕バンディッド問題の報酬関数`reward()`を実装して下さい。
    - 引数`record`は    
あらかじめ確保された総試行回数分のNumPy配列で、i回目の試行で得られた報酬の結果を配列中のi番目の位置に格納します。
<br><br>
    - 引数`results`は、    
`[[腕の番号, 選択回数, 報酬合計, 報酬合計 / 選択回数], [腕の番号, 選択回数, 報酬合計, 報酬合計 / 選択回数], [], []...]`  
の形の二重リスト構造です。
<br><br>
    - 引数`slot_num`は  
今回の試行で選択するスロットマシーンの番号(0,1,2,3,4のどれか)を示しています。  
<br><br>
    - 引数`time`は  
現在行なっている試行が何回目かを示す変数です。   
問題ではtime = 1000としているので、1000回目の試行をしたという意味になります。   

In [ ]:
import random
import numpy as np

random.seed(0)
np.random.seed(0)

# 環境の定義
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 関数rewardを作ってください
def reward(record, results, slot_num, time):
    # environments関数を用いて結果を取得してください
    result = 
    
    # recordのtime番目の値をresultにしてください
    
    
    # resultsの各値を更新してください
    
    
    
    
    return results, record

# 各変数の定義
times = 1000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
slot_num = 1
time = 1000

# 関数の呼び出し
results, record = reward(record, results, slot_num, time)

print(results)

#### ヒント

- 引数slot_numのスロットを引き、recordのtime番目の値と、resultsの各値を更新する関数を作りましょう。
- 1000番目の結果を反映させたresult,recordを更新してください。
- 1000回ではなく1000番目の結果であることに注意してください。

#### 解答例

In [ ]:
import random
import numpy as np

random.seed(0)
np.random.seed(0)

# 環境の定義
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 関数rewardを作ってください
def reward(record, results, slot_num, time):
    # environments関数を用いて結果を取得してください
    result = environments(slot_num)
    
    # recordのtime番目の値をresultにしてください
    record[time-1] = result
    
    # resultsの各値を更新してください
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    
    return results, record

# 各変数の定義
times = 1000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
slot_num = 1
time = 1000

# 関数の呼び出し
results, record = reward(record, results, slot_num, time)

print(results)

***

### 2.1.7 まとめ

以上ここまで
* 環境
* エージェント
* 報酬

という3つの言葉が出てきました。  
この3つの要素を数式で表現し、最適化していくことが強化学習の最も大切なフローです。

#### 問題

以下の条件でコードを追加し、平均報酬の推移をプロットして下さい。 
* 方策は毎回ランダムに選ぶものとします。
* 引く回数は100回とします。
* 腕の数は5本とします。
* それぞれの腕において当たりの出る確率は`np.array([0.3, 0.4, 0.5, 0.6, 0.7])`で定義されます。

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

np.random.seed(0)

# 手法を定義する関数です
def randomselect():
    slot_num = np.random.randint(0, 4)
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record

    
# 初期変数を設定しています
times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)

# slot_numを取得して、results,recordを書き換えてください
for time in range(0, times):
    slot_num = 
    results, record = 



# 各マシーンの試行回数と結果を出力しています
print(results)

# recordを用いて平均報酬の推移をプロットしてください


# 表を出力しています
plt.xlabel("試行回数")
plt.ylabel("平均報酬")
plt.title("試行回数と平均報酬の推移")
plt.show()


#### ヒント 

- `np.cumsum`関数によって配列の累積和を出すことができます。

####   解答例

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

np.random.seed(0)

# 手法を定義する関数です
def randomselect():
    slot_num = random.randint(0, 4)
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record


# 初期変数を設定しています
times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)

# slot_numを取得して、results,recordを書き換えてください
for time in range(0, times):
    slot_num = randomselect()
    results, record = reward(record, results, slot_num, time)

# 各マシーンの試行回数と結果を出力しています
print(results)

# recordを用いて平均報酬の推移をプロットしてください
plt.plot(np.cumsum(record) / np.arange(1, record.size + 1))

# 表を出力しています
plt.xlabel("試行回数")
plt.ylabel("平均報酬")
plt.title("試行回数と平均報酬の推移")
plt.show()


***

## 1.2 N腕バンディッド問題における方策

### 1.2.1 greedy手法

ここでは、N腕バンディッド問題をどのような**方策**で進めていくのが得策なのか議論しながら代表的な手法を紹介していきます。また実際に各手法を実装していき、それぞれの特徴について理解を深めましょう。

基本的な方針としては、最初のうちは確率に対する情報が全くないため、ランダムに腕を選ぶしかありません。試行を重ねるにつれて腕の成功率に関する情報が溜まってきます。そこでもっとも確率の高い腕を探し、その腕を選び続ける(**利用**と言います)選択を取ることとします。しかし試行回数が少ないうちは確率の偏りがあるため、一見成功率の低く見える腕も選択する(**探索**をする)必要があります。
 
最も単純な手法として、 **<font color=#AA0000>greedy手法</font>** という手法があります。基本的なアルゴリズムとしては、**これまでの結果から最も期待値の大きいスロットマシーンを選択する**というものになります。もちろんこれだけでは何も情報がない状態では判断することができないので、最初にある程度**探索**してデータを集める必要があります。N腕バンディッド問題でいえば各マシーンをn回動かすことが必要になります。

ここで実装するgreedyアルゴリズムを以下のように定義します。

1. まだn回選んだことのないスロットマシーンがある場合、それを選択する
2. 全てのマシンをn回選択した場合、全てのマシンに対してこれまでの報酬の期待値$u_{i}$を計算し、一番大きいものを選択する  
$u_{i}$ = これまでのマシンiの報酬の和 / これまでのマシンiのプレイ回数

#### 問題

- 現在の状況`result`、制限回数 ` n ` を引数として、次に引く腕を選ぶgreedy手法を実装して下さい。
- 引数`results`は、    
`[[腕の番号, 選択回数, 報酬合計, 報酬合計 / 選択回数], [腕の番号, 選択回数, 報酬合計, 報酬合計 / 選択回数], [], []...]`  
の形の二重リスト構造です。

In [ ]:
import numpy as np
np.random.seed(0)

# greedyアルゴリズムを実装して下さい
def greedy(results, n):
    # 試行回数がnより少ないマシンがある場合、slot_numをそのマシンの腕番号にする 
    # 以下のfor文内を埋めてください
    slot_num = None
    for i, d in enumerate(results):
        
        
        
    # どのマシンの試行回数もnより大きい場合、slot_numを報酬の期待値の高いものにする
    # 以下のif分内を埋めてください
    if slot_num == None:
        slot_num = 
        
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record


# 初期変数を設定して下さい
times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
n = 100

for time in range(0, times):
    slot_num = greedy(results,n)
    results, record = reward(record, results, slot_num, time)
    
print(results)


#### ヒント

- n回施行されていないマシンはif文で分岐して施行しましょう。

#### 解答例

In [ ]:
import numpy as np

np.random.seed(0)


# greedyアルゴリズムを実装して下さい
def greedy(results, n):
    # 試行回数がnより少ない場合、slot_numはそのままにする
    # 以下のfor文内を埋めてください
    slot_num = None
    for i, d in enumerate(results):
        if d[1] < n:
            slot_num = i
            break
        
    # 試行回数がnより大きい場合、slot_numを報酬の期待値の高いものにする
    # 以下のif分内を埋めてください
    if slot_num == None:
        slot_num = np.array([row[3] for row in results]).argmax()
        
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record

# 初期変数を設定して下さい
times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
n = 100

for time in range(0, times):
    slot_num = greedy(results,n)
    results, record = reward(record, results, slot_num, time)
    
print(results)

***

### 1.2.2 ε-greedy手法

先ほどの問題について考えてみましょう。  
nが少なくなればなるほど、greedy手法は間違ったマシン(確率が最大でないマシン)を選択する可能性が高くなります。かといってnを1000回にしても、高い精度で最適なスロットを決定できますが、それには5000回もの間最適でない手法でマシンを選択していることになりその分損をしていることがわかります。

このジレンマの解決方法としてよく知られた手法の一つに **<font color=#AA0000>ε-greedy手法</font>**(イプシロン-greedy手法)があります。**ε-greedy手法**とは利用と探索を**織り交ぜる**ことによって探索コストを減らしつつ、間違ったマシンを選択し続けるリスクを減らすことのできるgreedy手法の改善手法です。

**ε-greedy手法**を以下のように定義します。
 1. まだ選択したことがないマシンがある場合、それを選択する
 2. 確率εで全てのマシンの中からランダムに選択する(探索)
 3. 確率1-εでこれまでの報酬の平均が最大のマシンを選択する(利用)

εは我々が決める値です。(εは0より大きく、1以下の数です。)  
`ε = 0` でgreedy手法と等価になります  
このようにすることで、見積もりが不確かな場合でも期待値が高いマシンに選択を集中させることで無駄な探索コストを減らすと同時に、いつかは全ての腕が一定回数探索されるので間違えたマシンを選択するリスクも減らすことができます。  

#### 問題

ε-greedy手法を実装して下さい。
εの値を`0.01 0.1 0.2` と変化させてその収束の度合いをプロットしています。グラフをみて、εによる収束の違いを確認してください。

In [ ]:
import random
import numpy as np
from matplotlib import pyplot as plt

np.random.seed(0)

# ε-greedy手法を実装して下さい
def epsilon_greedy(results,epsilon):
    # 確率εで全てのマシンの中からランダムに選択してください(手順2)
    if np.random.binomial(1, epsilon):
        slot_num = 
        
    # 確率1-εでn=0のgreedy手法をとってください(手順1,3)
    else:
        slot_num =
        for i, d in enumerate(results):
            if d[1] == 0:
                slot_num = 
                break

        if slot_num == None:
            slot_num = 
            
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record

    
# 初期変数を設定しています
times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
epsilons = [0.01, 0.1, 0.2]

for epsilon in epsilons:
    for time in range(0, times):
        slot_num = epsilon_greedy(results, epsilon)
        results, record = reward(record, results, slot_num, time)

    #epsilon を変更させて収束の度合いをプロットしています
    plt.plot(np.cumsum(record) / np.arange(1, record.size + 1), label ="epsilon = %s" %str(epsilon))
    

#グラフと結果の出力をしています        
plt.legend()
plt.xlabel("試行回数")
plt.ylabel("平均報酬")
plt.title("ε-greedyのεによる収束の違い")
plt.show()
print(results)


#### ヒント

- アルゴリズムの手順1と手順3の内容はgreedy手法のものと全く同一です。

#### 解答例

In [ ]:
import random
import numpy as np
from matplotlib import pyplot as plt

np.random.seed(0)

# ε-greedy手法を実装して下さい
def epsilon_greedy(results,epsilon):
    # 確率εで全てのマシンの中からランダムに選択してください(手順2)
    if np.random.binomial(1, epsilon):
        slot_num = random.randint(0, 4)
        
    # 確率1-εでn=0のgreedy手法をとってください(手順1, 3)
    else:
        slot_num = None
        for i, d in enumerate(results):
            if d[1] == 0:
                slot_num = i
                break

        if slot_num == None:
            slot_num = np.array([row[3] for row in results]).argmax()
            
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record


    
# 初期変数を設定しています
times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
epsilons = [0.01, 0.1, 0.2]
for epsilon in epsilons:
    for time in range(0, times):
        slot_num = epsilon_greedy(results,epsilon)
        results, record = reward(record, results, slot_num, time)
        
    #epsilon を変更させて収束の度合いをプロットしています
    plt.plot(np.cumsum(record) / np.arange(1, record.size + 1), label ="epsilon = %s" %str(epsilon))

#グラフと結果の出力をしています
plt.legend()
plt.xlabel("試行回数")
plt.ylabel("平均報酬")
plt.title("ε-greedyのεによる収束の違い")
plt.show()
print(results)

***

### 1.2.3 楽観的初期値法

ここでgreedy手法のリスクについて考えてみましょう。

真の期待値がそれぞれ0.4、0.6である対象A、Bがあるとします。

**真の期待値が小さいものを、誤って期待値が大きいと予測した場合**（Aの期待値を0.8のように予測した時）は回数を重ねていくにつれ真の期待値に収束していきます。つまり、期待値が高いと予測したAを選択し続けることにより、Aの期待値の予想は0.4へと収束していくので、十分な回数の試行によって最終的にはBを選択するようになるのです。

しかし問題になるのは**真の期待値が大きいものを、誤って期待値が小さいと予測した場合**（Bの期待値を0.2のように予測したとき）です。このときは期待値が低いと予測したBを選択することはなく、最後まで期待値がより高い（0.4）と勘違いしたAだけを選び続けてしまうのです。

N腕バンディッド問題においていうならば、全ての腕をn回試行した段階で、真の確率が最も高い腕を選択できない可能性があるということです。

このような場合にはgreedy手法では修正が非常に困難であり、(真の期待値が0.4の方を選択し試行を続けるため、真の期待値が0.6の方が選択されないので修正するためのデータが蓄積せず、間違いに気づくことが困難です。

このようなリスクを減らすための原理として、「 **<font color=#AA0000>不確かな時は楽観的に</font>** 」という原理が知られています。つまりデータが蓄積されておらず、予測した期待値に不確実性があるときにはその期待値を**大きく見積もる**というものです。  

**楽観的初期値法**(optimistic initial rewards)はこのような発想をもとに、学習前に各腕から報酬の最大値をK回得ていたことを仮定する事で、不確かな状態での期待値を大きく見積もり、そこからgreedy法を適用する手法となります。具体的には、K=5だとすると、全てのマシンで1(当たり)をすでに5回引いているとしてから期待値を計算し始めることになります。<br>

楽観的初期値法のアルゴリズムは以下のように定義します。

1. 報酬の上界を$r_{sup}$で定義します。(スロット問題においては1)
2. すでに各マシンでK回上界が観測され、データとして蓄積しているとします。
3. この状態からgreedy法を始めます。

上界とは、とりうる最大値のことです。   


#### 問題

楽観的初期値法を実装して下さい。
変数の定義として
* $r_{sup}$は報酬の有界
* Kはすでに上界を観測した回数
* R(N)は実際に測定した報酬   
* Nは実際に測定した回数
* $\mu$(optimistic_mean)は報酬の期待値

$\mu=\frac{R(N)+Kr_{sup}}{N + K}$

であるものとします。



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

# 楽観的初期値法を実装して下さい
def optimistic(results, K, rsup):
    # スロットごとの報酬の期待値を計算して、配列に格納してください
    optimistic_mean = 
    # 最も報酬の期待値が大きいスロットを選択してください
    slot_num = 
    return slot_num

# (参考)１つ前のchapterで実装したepsilon_greedy法です
def epsilon_greedy(results,epsilon):
    if np.random.binomial(1, epsilon):
        slot_num = random.randint(0, 4)
    else:
        slot_num = None
        for i, d in enumerate(results):
            if d[1] == 0:
                slot_num = i
                break
        if slot_num == None:
            slot_num = np.array([row[3] for row in results]).argmax()     
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record

# 初期変数を設定しています
times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
Ks = range(1,6)

# Kを変更させて収束の度合いをプロットしています
for K in Ks:
    for time in range(times):
        slot_num = optimistic(results, K, 1)
        results, record = reward(record, results, slot_num, time)
    plt.plot(np.cumsum(record) / np.arange(1, record.size + 1), label ="K = %d" % K)
plt.legend()
plt.xlabel("試行回数")
plt.ylabel("平均報酬")
plt.title("楽観的初期値法のKの変動による結果の差異")
plt.show()


#### ヒント

 - results変数が最初から各マシンにおいてK回1が当たり続けた結果を格納しているものとして考えましょう。
 - optimistic_mean = np.array([(row[2] + K*rsup) / (row[1] + K) for row in results])

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

# 楽観的初期値法を実装して下さい
def optimistic(results, K, rsup):
    # スロットごとの報酬の期待値を計算して、配列に格納してください
    optimistic_mean = np.array([(row[2] + K*rsup) / (row[1] + K) for row in results])
    # 最も報酬の期待値が大きいスロットを選択してください
    slot_num = optimistic_mean.argmax()
    return slot_num

# (参考)１つ前のchapterで実装したepsilon_greedy法です
def epsilon_greedy(results,epsilon):
    if np.random.binomial(1, epsilon):
        slot_num = random.randint(0, 4)
    else:
        slot_num = None
        for i, d in enumerate(results):
            if d[1] == 0:
                slot_num = i
                break
        if slot_num == None:
            slot_num = np.array([row[3] for row in results]).argmax()     
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record

# 初期変数を設定しています
times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
Ks = range(1,6)

# Kを変更させて収束の度合いをプロットしています
for K in Ks:
    for time in range(times):
        slot_num = optimistic(results, K, 1)
        results, record = reward(record, results, slot_num, time)
    plt.plot(np.cumsum(record) / np.arange(1, record.size + 1), label ="K = %d" % K)
plt.legend()
plt.xlabel("試行回数")
plt.ylabel("平均報酬")
plt.title("楽観的初期値法のKの変動による結果の差異")
plt.show()


***

### 1.2.4 soft-max法

ε-greedy手法は確かに有用な手法ですが、探査を行う際にすべての行動を等しい確率で選択してしまう点が欠点です。  
つまりほぼ最悪と思われる選択を取る確率とほぼ最適と思われる選択を取る確率が同程度になっているのです。例えばスロットマシーンの中に
* 当たりだと1
* 外れだと-100
* 当たる確率は20%

のものがあったとしましょう。このような極めて悪い行動があった場合、ε-greedy手法ではこれを少ない回数選択するということができません。そこで **<font color=#AA0000>soft-max</font>** 法では、推定される行動から価値が高そうな行動はより選ばれやすく、低そうな行動はより選ばれににくくします。ただし、全く選ばれなくなるわけではありません。正確に述べると、soft-max法とは**推定報酬によって選択する確率に重み付けをすることが可能になる手法**です。具体的には、 $Q_i$ を報酬の期待値、$\tau$ (タウ)をパラメータとして$\frac{\exp{Q_{i}/ \tau}}{\sum^i \exp{Q_{i}/ \tau}}$と確率を表すことができます。$\tau$が高ければすべての行動が選ばれるようになり、0に近づくと推定値の価値が高い行動が選ばれやすくなります。また、$\tau$→0 の時、greedy手法と等価になります。

soft-maxアルゴリズムは以下のように定義されます。
1. 今までのデータがない場合、全ての手法の報酬を1で仮定します。
2. 各マシンiの選択確率を$\frac{\exp{Q_{i}/ \tau}}{\sum_i \exp{Q_{i}/ \tau}}$で定義します。
3. 2で定義した確率分布に基づき、選択を行います
4. 選択で得られた報酬に基づき、報酬関数を更新します。

#### 問題

soft-max手法を実装して下さい。

In [ ]:
import numpy as np
# 回答確認用です。変えないでください
np.random.seed(55)

tau = 0.1

# soft-max法を実装して下さい
def softmax(results, tau):
    # 各マシンの予想期待値を配列にしてください
    q = 
    
    # 今までのデータがない場合、全ての手法の報酬を1で仮定してください
    if np.sum(q) == 0:
        q = 
        
    # 各マシンの選択確率を設定してください
    probability = 
    
    # 確率分布に基づいて、slot_numを決定してください
    slot_num = 
    
    return slot_num



# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record


results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
times = 1000
record = np.zeros(times)
for time in range(0, times):
    slot_num = softmax(results,tau)
    results, record = reward(record, results, slot_num, time)
print(results)


#### ヒント

- 選択に`np.random.choice(配列, p=出現確率)`関数を使用してみましょう。
- $e^0$ = 1 であるので $Q_i$=0でも確率は0にはなりません。

#### 解答例

In [ ]:
import numpy as np
# 回答確認用です。変えないでください
np.random.seed(55)

tau = 0.1

# soft-max法を実装して下さい
def softmax(results, tau):
    # 各マシンの予想期待値を配列にしてください
    q = np.array([row[3] for row in results])
    
    # 今までのデータがない場合、全ての手法の報酬を1で仮定してください
    if np.sum(q) == 0:
        q = np.ones(len(results))
        
    # 各マシンの選択確率を設定してください
    probability = np.exp(q / tau) / sum(np.exp(q / tau))
    
    # 確率分布に基づいて、slot_numを決定してください
    slot_num = np.random.choice([row[0] for row in results], p=probability)
    
    return slot_num


# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record


results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
times = 1000
record = np.zeros(times)
for time in range(0, times):
    slot_num = softmax(results,tau)
    results, record = reward(record, results, slot_num, time)
print(results)


***

### 1.2.5 UCB1アルゴリズム


 **<font color=#AA0000>UCB1アルゴリズム</font>** は楽観的初期値法を改善した手法です。  
greedy手法などで見られたような今までのデータから導出される期待値にバイアス(試行回数が少ない場合に大きくなる)を加えて最大となるマシンを選択します。<br>
これはつまり、**そのマシンがどれほど当たってきたか**(成功率)という情報に**そのマシンについてどれだけ知っているか**(偶然による成功率のばらつきの大きさ)という情報を付け加えていくということに等しくなっています。こうすることによってあまり探索されていないマシンを積極的に探索するとともに、データが集まってくるにつれて最も当選確率の高いと見られるマシンを選択するということが同時にできるようになります。

以下のようにしてUCB1アルゴリズムは定義されます。
> 1. R : 報酬の最大値と最小値の差(今回は1です)とします。 
> 2. まだ選んだことのないマシンがあればそれを選択する。
> 3. マシンごとに今までの結果から得られた報酬の期待値$u_{i}$(腕の成功率)を計算する。   
> <img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/equation_1.png">
> 4. マシンごとの偶然による成功率のばらつきの大きさである$x_{i}$を計算する。  
> <img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/equation_2.png">
> 5. $UCB1 = u_{i} + x_{i}$が最大値になるマシンiを選択してプレイする。


#### 問題

- 空欄を埋めてUCB手法を実装して下さい。

In [ ]:
import numpy as np
import math
np.random.seed(0)

R = 1

#　UCB1法を実装して下さい
def UCB(results, R):
    slot_num = None
    # まだ選んだことのないマシンがあればそれをslot_numにしてください
    for i, d in enumerate(results):
        
        
        
        
    # 全て一度は選んでいる場合
    if slot_num == None:
        
        # これまでの総試行回数を計算してください
        times = 
        
        # マシンごとに今までの結果から得られた成功率uiを取り出してリストに格納してください
        ui = 
        
        # マシンごとの偶然による成功率のばらつきの大きさxi を計算してリストに格納してください
        xi = 
        
        # ui+xiを計算してリストに格納してください
        uixi = 
        
        # uixiが最大値となるマシンをslot_numにしてください
        slot_num = 
        
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record


results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
times = 1000
record = np.zeros(times)
for time in range(0, times):
    slot_num = UCB(results,R)
    results, record = reward(record, results, slot_num, time)
print(results)

#### ヒント

- 一定のプレイ回数に至るまで各腕をプレイし、配列から最大値を記録するインデックスを抽出する事は今までの手法と同じです。

#### 解答例

In [ ]:
import numpy as np
import math
np.random.seed(0)

R = 1

#　UCB1法を実装して下さい
def UCB(results, R):
    slot_num = None
    # まだ選んだことのないマシンがあればそれをslot_numにしてください
    for i, d in enumerate(results):
        if d[1] == 0:
            slot_num = i
            break

    # 全て一度は選んでいる場合
    if slot_num == None:
        
        # これまでの総試行回数を計算してください
        times = sum([row[1] for row in results])
        
        # マシンごとに今までの結果から得られた成功率uiを取り出してリストに格納してください
        ui = [row[3] for row in results]
        
        # マシンごとの偶然による成功率のばらつきの大きさxi を計算してリストに格納してください
        xi = [R*math.sqrt(2*math.log(times) / row[1]) for row in results]
        
        # ui+xiを計算してリストに格納してください
        uixi = [x+u for x, u in zip(xi, ui)]
        
        # uixiが最大値となるマシンをslot_numにしてください
        slot_num = uixi.index(max(uixi))
        
    return slot_num

# 環境を定義する関数です
def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

# 報酬を定義する関数です
def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record


results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
times = 1000
record = np.zeros(times)
for time in range(0, times):
    slot_num = UCB(results,R)
    results, record = reward(record, results, slot_num, time)
print(results)

***

### 1.2.6 まとめ

これまで
N腕バンディッド問題を解くための方策として
- greedy手法
- ε-greedy手法
- soft-max手法
- 楽観的初期値法
- UCB1アルゴリズム  

の5つについて説明してきました。ここで一つ強調しておくべきは、ある一つの問題について各手法を比較しても、手法の優劣を議論する根拠にはならないということです。重要なのはこれらの手法の間に明確な優劣があるというわけではなく、問題設定に応じて最適な手法を使い分ける必要があるということです。このことをまとめとして覚えておいて下さい。

＜備考＞<br>
最初から全ての成功率がわかっているときは、もっとも成功率の高いものを選び続ければ良いことになります。この場合とある方策との報酬の合計の差分のことを**リグレット**と呼びます。エージェントは必ず探索を行わないといけなため、リグレットを0にすることは不可能ですがある程度試行回数を重ねれば次第にリグレットは小さくなっていきます。<br>
**UCB1アルゴリズムはこのリグレットを最小化することができることが知られています。**

#### 問題

- (80行目以降)greedy法、ε-greedy手法、楽観的初期値法、UCB1アルゴリズムの4つの手法の結果をそれぞれプロットし、それぞれの手法における精度を確認してください。

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math

np.random.seed(0)

def greedy(results, n):
    slot_num = None
    for i, d in enumerate(results):
        if d[1] < n:
            slot_num = i

    if slot_num == None:
        slot_num = [row[3] for row in results].index(max([row[3] for row in results]))
    return slot_num


def optimistic(results, K, rsup):
    optimistic_mean = [(row[2] + K*rsup) / (row[1] + K) for row in results]
    slot_num = optimistic_mean.index(max(optimistic_mean))
    return slot_num


def epsilon_greedy(results, epsilon):
    if np.random.binomial(1, epsilon):
        slot_num = random.randint(0, 4)
    else:
        slot_num = None
        for i, d in enumerate(results):
            if d[1] == 0:
                slot_num = i
        if slot_num == None:
            slot_num = np.array([row[3] for row in results]).argmax()           
    return slot_num


def UCB(results, R):
    slot_num = None
    for i, d in enumerate(results):
        if d[1] == 0:
            slot_num = i
    if slot_num == None:
        times = sum([row[1] for row in results])
        u = [row[3] for row in results]
        xi = [R*math.sqrt(2*math.log(times) / row[1]) for row in results]
        uixi = [x+u for x, u in zip(xi, u)]
        slot_num = uixi.index(max(uixi))
    return slot_num


def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result


def reward(record, results, slot_num, time):
    result = environments(slot_num)
    
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2]/results[slot_num][1]
    return results,record


times = 10000
n = 20
K = 10
rsup = 1
R = 1
epsilon = 0.2
results = [[[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]] for _ in range(4)]
slot_num = np.zeros(4,dtype=np.int)
record = np.zeros((4,times))
# この続きをコーティングして手法ごとにプロットしてください

# resultsからそれぞれの方策でslot_numを決定してください
for time in range(0, times):
    # greedy手法
    slot_num[0] = 
    # ε-greedy手法
    slot_num[1] = 
    # 楽観的初期値法
    slot_num[2] = 
    # UCB1アルゴリズム
    slot_num[3] = 
    
    # results、recordを上書きしてください
    for i in range(4):
        results[i], record[i] = 

# 凡例を以下のように設定しています
labels = ["greedy","epsilon_greedy","optimistic","UCB"]
for i in range(4):
    # 4種の手法の結果をプロットしてください
    

plt.legend(loc="lower right")
plt.xlabel("試行回数")
plt.ylabel("平均報酬")
plt.title("N腕バンディッド問題の各手法における報酬の収束")
plt.show()


#### ヒント

- results, slot_num, recordはすべて今までのものとは別の形で初期化されているので注意してください。
- xi = [R*math.sqrt(2*math.log(times) / row[1]) for row in results]

#### 解答例

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
np.random.seed(0)


def greedy(results, n):
    slot_num = None
    for i, d in enumerate(results):
        if d[1] < n:
            slot_num = i

    if slot_num == None:
        slot_num = [row[3] for row in results].index(max([row[3] for row in results]))
    return slot_num


def optimistic(results, K, rsup):
    optimistic_mean = [(row[2] + K*rsup) / (row[1] + K) for row in results]
    slot_num = optimistic_mean.index(max(optimistic_mean))
    return slot_num


def epsilon_greedy(results, epsilon):
    if np.random.binomial(1, epsilon):
        slot_num = random.randint(0, 4)
    else:
        slot_num = None
        for i, d in enumerate(results):
            if d[1] == 0:
                slot_num = i
        if slot_num == None:
            slot_num = np.array([row[3] for row in results]).argmax()           
    return slot_num


def UCB(results, R):
    slot_num = None
    for i, d in enumerate(results):
        if d[1] == 0:
            slot_num = i
    if slot_num == None:
        times = sum([row[1] for row in results])
        u = [row[3] for row in results]
        xi = [R * math.sqrt(2 * math.log(times) / row[1]) for row in results]
        uixi = [x + u for x, u in zip(xi, u)]
        slot_num = uixi.index(max(uixi))
    return slot_num


def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result


def reward(record, results, slot_num, time):
    result = environments(slot_num)

    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record


times = 10000
n = 20
K = 10
rsup = 1
R = 1
epsilon = 0.2
results = [[[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]] for _ in range(4)]
slot_num = np.zeros(4,dtype=np.int)
record = np.zeros((4,times))
# この続きをコーティングして手法ごとにプロットしてください

# resultsからそれぞれの方策でslot_numを決定してください
for time in range(0, times):
    # greedy手法
    slot_num[0] = greedy(results[0],n)
    # ε-greedy手法
    slot_num[1] = epsilon_greedy(results[1],epsilon)
    # 楽観的初期値法
    slot_num[2] = optimistic(results[2], K, rsup)
    # UCB1アルゴリズム
    slot_num[3] = UCB(results[3], R)
    
    # results、recordを上書きしてください
    for i in range(4):
        results[i], record[i] = reward(record[i], results[i], slot_num[i], time)

# 凡例を以下のように設定しています
labels = ["greedy","epsilon_greedy","optimistic","UCB"]
for i in range(4):
    # 4種の手法の結果をプロットしてください
    plt.plot(np.cumsum(record[i]) / np.arange(1, record[i].size + 1),label=labels[i])
    
plt.legend(loc="lower right")
plt.xlabel("試行回数")
plt.ylabel("平均報酬")
plt.title("N腕バンディッド問題の各手法における報酬の収束")
plt.show()

***

### 1.2.7 探索と利用のトレードオフ

 **<font color=#AA0000>探索と利用のトレードオフ</font>** について説明していきます。

これは、探索と利用のバランスの問題であり、探索の割合を増やすと、探索中は最善の選択を選ばないため、そのぶん損失が発生してしまうのに対し、利用の割合を増やすと、最善の選択が今までされてこなかった場合にそれを見落とすリスクが発生してしまいます。このようにトレードオフの関係になっているのです。例えばε-greedy手法とUCB手法について比較してみましょう。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/img_3.png">


それぞれの方策に従って100000回引き続けるシミュレーションです。8000回前後辺りまではε-greedy手法の方が高い平均報酬率を示していますが、回数が増えるにつれUCB手法の方が上回るようになります。それはε-greedy手法が十分に探索を終えた後でも一定確率で探索を行ってしまい、それが損失になっているからなのです。  

この**トレードオフの関係を環境の状態によってどのように取るかが強化学習の本質**とも言えます。

#### 問題

以下のうち正しいものはどれでしょう？

- 正しい期待値を得るため探索回数は多いほうがいい
- コストを抑えるため探索回数はある程度に抑える
- コストは抑えて期待値の最も高い試行を実行する

#### ヒント

- トレードオフの関係にあるものは両方のいいとこどりはできません。

#### 解答

- コストを抑えるため探索回数はある程度に抑える

***

## 添削問題

#### 問題

nを2回、10回、200回というように変えて、  
greedy手法を使用して4000回引くシミュレーションを行い、平均報酬率の変化を描画してください。 

これを10回行い平均報酬率を計算して下さい。


In [ ]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import math


def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

def reward(record, results, slot_num, time):
    result = environments(slot_num)

    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2]/results[slot_num][1]
    return results, record

#greedy法を実装してください。
def greedy(results, n):

    
    
    return slot_num



# 与えられたn、総試行回数に基づいて一度実行し、plt.plot()の返り値と平均報酬率を返します
# 答えを入力して下さい
def nsearch(n, times):
  




    return p1, average

# 初期化します
#averagesにはn=2,10,200のときの平均報酬率をリストにして格納してください。
plt.clf()  
times = 4000
averages = [0, 0, 0]


# 最初の結果をaveragesに格納しプロットします　答えを入力して下さい




plt.show()

# 残り9回を行いaveragesに格納します　答えを入力して下さい
for i in range(9):
    
    
    
    
    
# 答えを出力します　
answer = list(map(lambda x: x/10, averages))
print(answer)

#### ヒント

平均報酬率の計算にはrecordの累積和を使いましょう。

#### 解答例

In [ ]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import math

def environments(band_number):
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result


def reward(record,results,slot_num,time):
    result = environments(slot_num)

    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2]/results[slot_num][1]
    return results, record

def greedy(results, n):
    slot_num = None
    for i, d in enumerate(results):
        if d[1] < n:
            slot_num = i

    if slot_num == None:
        slot_num = [row[3] for row in results].index(max([row[3] for row in results]))
    return slot_num



# 与えられたn、総試行回数に基づいて一度実行し、plt.plot()の返り値と平均報酬率を返します
# 答えを入力して下さい
def nsearch(n, times):
    results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
    record = np.zeros(times)
    for time in range(times):
        slot_num = greedy(results, n)
        
        results, record = reward(record, results, slot_num, time)

        
    p1, = plt.plot(np.cumsum(record) / np.arange(1, record.size + 1))
    average = (np.sum(record))/(record.size)
    
    return p1, average

# 初期化します
plt.clf()  
times = 4000
averages = [0, 0, 0]


# 最初の結果をaveragesに格納しプロットします　答えを入力して下さい
p1, averages[0] = nsearch(2, times)
p2, averages[1] = nsearch(10, times)
p3, averages[2] = nsearch(200, times)
plt.legend([p1, p2, p3], ["2", "10", "200"])
plt.xlabel("times")
plt.ylabel("result")
plt.show()

# 残り9回を行いaveragesに格納します　答えを入力して下さい
for i in range(9):
    averages[0] += (nsearch(2, times))[1] 
    averages[1] += (nsearch(10, times))[1] 
    averages[2] += (nsearch(200, times))[1]

    
# 答えを出力します　
answer = list(map(lambda x: x/10, averages))
print(answer)

***